In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

/home/arjun/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# TODO
- See if ticket and cabin have any importance
- Try data imputation instead of fillna

# Dataset

In [2]:
loc = '/home/arjun/Desktop/Datasets/titanic'

In [3]:
df = pd.read_csv(loc + '/train.csv')

In [4]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [6]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})

In [7]:
df.fillna(df.mean(), inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,1.0
1,2,1,1,1,38.0,1,0,71.2833,2.0
2,3,1,3,1,26.0,0,0,7.9250,1.0
3,4,1,1,1,35.0,1,0,53.1000,1.0
4,5,0,3,0,35.0,0,0,8.0500,1.0


In [8]:
ds = torch.tensor(np.array(df))
df.shape, ds.shape

((891, 9), torch.Size([891, 9]))

In [9]:
ds.shape

torch.Size([891, 9])

In [10]:
y = ds[:,2]
X= torch.cat((ds[:,:2], ds[:,3:]), dim=-1) 
X.shape

torch.Size([891, 8])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_train.shape, X_test.shape ,y_train.shape, y_test.shape

(torch.Size([712, 8]),
 torch.Size([179, 8]),
 torch.Size([712]),
 torch.Size([179]))

# Model Creation

In [12]:
num_epochs = 100
lr =.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/home/arjun/AI_ENV/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [13]:
class NeuralNet(nn.Module):
    def __init__(self, inp_size, h1, h2, h3, h4, h5, out_size):
        super(NeuralNet, self).__init__()
        self.inp_size = inp_size
        self.lay1 = nn.Linear(inp_size, h1)
        self.lay2 = nn.ReLU()
        self.lay3 = nn.Linear(h1, h2)
        self.lay4 = nn.ReLU()
        self.lay5 = nn.Linear(h2, h3)
        self.lay6 = nn.ReLU()
        self.lay7 = nn.Linear(h3, h4)
        self.lay8 = nn.ReLU()
        self.lay9 = nn.Linear(h4, h5)
        self.lay10 = nn.ReLU()
        self.lay11 = nn.Linear(h5, out_size)
        
    def forward(self,x):
        out = self.lay1(x)
        out = self.lay2(out)
        out = self.lay3(out)
        out = self.lay4(out)
        out = self.lay5(out)
        out = self.lay6(out)
        out = self.lay7(out)
        out = self.lay8(out)
        out = self.lay9(out)
        out = self.lay10(out)
        out = self.lay11(out)  # We don't apply sotmax the cross entropy loss will do that for us
        return out
    
model = NeuralNet(8,30,60,120,60,30,2).to(device)

In [14]:
# class Dataset:
#     def __init__(self, X, y):
#         self.xy = 
#         self.X = X
#         self.y = y

In [15]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr = lr)

# Training Model

In [16]:
train_ds = X = torch.cat((X_train, y_train.reshape(-1,1)), dim=-1)  
train_ds[:, -1]

tensor([3., 3., 3., 3., 1., 3., 2., 2., 3., 3., 1., 3., 2., 3., 2., 1., 3., 3.,
        2., 3., 3., 2., 2., 3., 3., 3., 3., 2., 1., 3., 3., 3., 1., 3., 3., 3.,
        3., 3., 2., 2., 1., 1., 3., 3., 1., 3., 3., 2., 1., 3., 1., 2., 3., 3.,
        1., 3., 3., 3., 2., 2., 2., 3., 2., 3., 3., 3., 2., 3., 2., 2., 3., 3.,
        1., 3., 1., 3., 3., 2., 2., 3., 3., 3., 3., 2., 3., 3., 3., 2., 3., 1.,
        3., 2., 2., 3., 1., 3., 3., 1., 3., 1., 2., 1., 1., 1., 3., 1., 3., 3.,
        3., 3., 3., 3., 1., 1., 1., 3., 3., 3., 3., 3., 1., 3., 2., 2., 2., 1.,
        1., 3., 3., 3., 3., 2., 1., 1., 3., 2., 3., 3., 3., 3., 1., 3., 2., 2.,
        3., 1., 2., 3., 3., 1., 1., 3., 3., 2., 3., 3., 1., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 1., 3., 1., 3., 3., 1., 1., 2., 3., 1., 2.,
        3., 3., 2., 3., 3., 2., 3., 3., 3., 3., 2., 3., 3., 1., 3., 3., 3., 3.,
        2., 3., 3., 2., 3., 3., 2., 1., 3., 1., 3., 3., 3., 2., 1., 2., 2., 3.,
        2., 3., 3., 3., 1., 3., 2., 1., 

In [17]:
train_loader = torch.utils.data.DataLoader(dataset=train_ds, shuffle=True, batch_size=100)


In [21]:
for epoch in range(num_epochs):
    avg_loss = 0
    for i,batch in enumerate(train_loader):

        y = F.one_hot(batch[:,-1], num_classes=2).to(device).long()
        X = batch[:, :-1].to(device).to(torch.float32)

        # Forward pass
        output = model(X)
        print(output, y)
        
        # Backward pass
        loss = lossCategory(output, y)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        avg_loss += loss

        
    print(f"Epoch:{epoch+1}/{num_epochs} Loss:{avg_loss/len(train_loader)}")

RuntimeError: one_hot is only applicable to index tensor.

# Checking Accuracy

In [ ]:
# correct = 0
# for i,batch in enumerate(train_loader):
#     y = batch[:,1].to(device).long().item()
#     X =torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
#     output = model(X)
#     softmax_output = F.softmax(output, dim=-1)
#     y_pred = torch.argmax(softmax_output).item()
#     if y_pred == y:
#         correct += 1
# print('Accuracy:', correct*100/i) 

correct = 0
tot = 0
for i,batch in enumerate(train_loader):
    
    y = batch[:,1].to(device).long()
    X =torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
    y_pred = torch.argmax(model(X), dim=1)
    s = sum(y_pred == y)
    correct += s
    tot += len(y)
    
print('Accuracy:', (correct*100/tot).item()) 

# Running Test Dataset

In [ ]:
df = pd.read_csv(loc + '/test.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})
df.fillna(df.mean(), inplace=True)
ds = torch.tensor(np.array(df))
test_loader = torch.utils.data.DataLoader(dataset=ds, shuffle=False, batch_size=1)

In [ ]:
ds.shape,df.shape

In [ ]:
l = []
id_ = []
for i,batch in enumerate(test_loader):
        X =batch.to(device).to(torch.float32)
        output = model(X)
        softmax_output = F.softmax(output, dim=-1)

        l.append(torch.argmax(softmax_output).item())
        id_.append(int(X[:,0].item()))

ans = pd.DataFrame({'PassengerId':id_,'Survived':l})
ans.to_csv('./submission.csv', index=False)
print("Saved new O/P")
ans